In [113]:
import yfinance as yf
import datetime as dt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

america = (dt.datetime.today() - pd.Timedelta(days=2)).date().strftime("%Y-%m-%d")
world = (dt.datetime.today() - pd.Timedelta(days=1)).date().strftime("%Y-%m-%d")
nyse_open = pd.to_datetime(f"{america}T13:30:00", utc=True)
nyse_close = pd.to_datetime(f"{america}T20:00:00", utc=True)
asx_open = pd.to_datetime(f"{world}T00:00:00", utc=True)
asx_close = pd.to_datetime(f"{world}T06:00:00", utc=True)
jpx_open = pd.to_datetime(f"{world}T00:00:00", utc=True)
jpx_close = pd.to_datetime(f"{world}T06:00:00", utc=True)
hs_open = pd.to_datetime(f"{world}T01:30:00", utc=True)
hs_close = pd.to_datetime(f"{world}T08:00:00", utc=True)

In [114]:
def plot_price_and_volume(df: pd.DataFrame, name: str):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Price", "Volume"))

    fig.add_trace(
        go.Scatter(x=df.index, y=(df["High"] + df["Low"])/2, name=f"{name}"),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(x=df.index, y=df["Volume"], name="Volume"),
        row=2, col=1
    )
    fig.update_layout(title=f"{name}", yaxis_title="Price ($)", yaxis2_title="Volume")
    return fig

def plot_price(df: pd.DataFrame, name: str, currency: str):
    return px.line(x=df.index, y=(df["High"] + df["Low"])/2, title=f"{name}", labels={"y": f"Price ({currency})"})

def headlines(df: pd.DataFrame):
    res = []
    for x in df.news:
        res.append(x["title"])

    display(res)

def percentage(x: int, y: int):
    return round(((y - x) / x) * 100, 2)

In [115]:
asx_200 = yf.Ticker("^AXJO")
asx_200_yest = asx_200.history(start=asx_open, end=asx_close, interval="1m")

s_and_p = yf.Ticker("^GSPC")
s_and_p_yest = s_and_p.history(start=nyse_open, end=nyse_close, interval="1m")

jpx = yf.Ticker("^N225")
jpx_yest = jpx.history(start=jpx_open, end=jpx_close, interval="1m")

hs = yf.Ticker("^HSI")
hs_yest = hs.history(start=hs_open, end=hs_close, interval="1m")

vix = yf.Ticker("^VIX")
vix_day = vix.history(period="1d", interval="1m")

# Daily Markets Report 

## Summary

In [116]:
summary = [asx_200, s_and_p, jpx, hs]
hist = [asx_200_yest, s_and_p_yest, jpx_yest, hs_yest]
obs = {
    "Index": [],
    "Prev. Close to Open": []
}
for i, x in enumerate(summary):
    obs["Index"].append(x.info["longName"])
    obs["Prev. Close to Open"].append(percentage(x.info["open"], x.info["previousClose"]))

pd.DataFrame(obs).set_index("Index")

,Prev. Close to Open
Index,
S&P/ASX 200,0.00
S&P 500,-0.39
Nikkei 225,-1.93
HANG SENG INDEX,-0.97


### VIX

In [117]:
headlines(vix)

['The 5 reasons behind Monday’s global stock market rout',
 'S&P 500, Dow Mark Worst Day Since 2022',
 'Your stocks now trade at the mercy of the Japanese yen',
 'The global market meltdown just sent Wall Street’s ‘fear gauge’ to a 4-year high',
 'Five Fed Rate Cuts? Treasury Yields Dive As Volatility Soars On Bad Weather, Yen, Iran',
 'Stocks Plunge at the Open. Volatility Spikes.',
 "Wall Street 'fear gauge' logs record jump as investors bet on more turbulence",
 'Stock Market News for Aug 5, 2024']

In [118]:
plot_price(vix_day, "VIX", "Volatility")

## USA

### S&P 500

#### Headlines

In [119]:
headlines(s_and_p)

['Wall Street watching Fed, Google ruling, Palantir: 3 Things',
 'Prediction: Crypto Prices Will Surge After the Sell-Off, Seize Your Chance to Buy Now',
 'Stocks Bounce Back After Selloff as Bonds Retreat: Markets Wrap',
 'Got $1,000 to Invest In Stocks? Put It In This Index Fund.',
 'Market Sell-Off Loses Steam as US Equity Futures Turn Higher Pre-Bell',
 'Stifel says dip is not a blip, stay defensive',
 'Heard on the Street: Prepare for a Volatility Relapse',
 'Vertical Violations Can Help Avoid Bear Market']

#### Price Information

In [120]:
s_and_p_yest.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,388.000000,388.000000,388.000000,388.000000,3.880000e+02,388.0,388.0
mean,5197.695602,5200.438659,5195.081855,5197.735436,8.284823e+06,0.0,0.0
std,23.434240,22.579083,23.631332,23.078013,6.594579e+06,0.0,0.0
min,5120.580078,5125.669922,5119.259766,5121.080078,0.000000e+00,0.0,0.0
25%,5178.945190,5182.027588,5175.804810,5178.495117,5.038250e+06,0.0,0.0
50%,5200.535156,5202.130127,5198.159912,5200.324951,6.488500e+06,0.0,0.0
75%,5215.505005,5217.869873,5213.425171,5215.312500,9.158000e+06,0.0,0.0
max,5250.759766,5250.890137,5248.419922,5250.740234,8.575700e+07,0.0,0.0


In [121]:
plot_price(s_and_p_yest, "S&P 500", "$USD")

## Australia

### ASX 200

#### Headlines

In [122]:
headlines(asx_200)

['Central Banks Caught Between Inflation and Market Fears',
 'RBA no longer expected to hike rates in 2024, but cuts also unlikely- Rabobank',
 'Australia trade balance grows more than expected in June',
 'Australia jobs surge past expectations in June, labor market remains tight',
 'Australia CPI up more than expected in May at 4%, RBA jitters mount',
 'Australia jobs grow more than expected in May, unemployment eases',
 'Australia trade balance grows more than expected in April as imports shrink',
 'Australia GDP disappoints in Q1 on pressure from inflation, interest rates']

#### Price Information

In [123]:
asx_200_yest.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,356.000000,356.000000,356.000000,356.000000,356.0,356.0,356.0
mean,7675.843274,7676.620232,7675.117418,7675.894376,0.0,0.0,0.0
std,15.828092,15.712140,15.830569,15.750582,0.0,0.0,0.0
min,7628.100098,7630.299805,7628.100098,7630.299805,0.0,0.0,0.0
25%,7669.899902,7670.399902,7669.500000,7669.949951,0.0,0.0,0.0
50%,7674.550049,7675.449951,7673.650146,7674.550049,0.0,0.0,0.0
75%,7683.500000,7684.225098,7682.824829,7683.525024,0.0,0.0,0.0
max,7713.700195,7713.700195,7710.000000,7710.500000,0.0,0.0,0.0


In [124]:
plot_price(asx_200_yest, "ASX 200", "$AUD")

## Japan

#### Headlines

In [125]:
headlines(jpx)

['How the global sell-off is impacting currencies',
 'Nikkei Rallies and Japanese Tech Stocks Recover',
 'Heard on the Street: Prepare for a Volatility Relapse',
 'Nikkei Stock Average Has Best Day Since 2008',
 'Asian stocks rebound from steep losses; Nikkei surges over 10%',
 "Japan's Nikkei 225 stock index plunges 12.4% as investors dump a wide range of shares",
 'Why a Rising Yen Is One More Headache for Global Markets',
 'Dark Day for Japan Stocks as Unease About U.S. Spreads']

#### Price Information

In [126]:
jpx_yest.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,302.000000,302.000000,302.000000,302.000000,302.0,302.0,302.0
mean,34268.173000,34308.631739,34232.716441,34274.707717,0.0,0.0,0.0
std,443.532256,423.955577,442.797790,424.789960,0.0,0.0,0.0
min,32077.330078,32086.089844,32077.330078,32086.089844,0.0,0.0,0.0
25%,34051.443359,34084.379883,34016.816406,34052.262695,0.0,0.0,0.0
50%,34312.369141,34339.394531,34266.425781,34322.544922,0.0,0.0,0.0
75%,34588.037109,34627.790039,34551.238281,34588.905273,0.0,0.0,0.0
max,34901.550781,34911.800781,34883.109375,34887.480469,0.0,0.0,0.0


In [127]:
plot_price(jpx_yest, "Nikkei 400", "¥")

## Hong Kong

#### Headlines

In [128]:
headlines(hs)

['Japan Leads Hopes for Global Market Rebound. Most Gains Still Aren’t Much.',
 'Japan Rebound Leads Asian Stock Markets Out of Rout',
 'Japanese stocks rebound from worst crash since 1987 while global markets are mixed',
 'The Unexpected Winners and Losers as Markets Went\xa0Into\xa0Meltdown',
 'Japan stocks rebound after worst crash since 1987',
 'Asian stocks rebound from steep losses; Nikkei surges over 10%',
 "Wall Street points toward gains as calm returns to markets; Japan's Nikkei roars back to gain 10%",
 'Bets on emergency US rate cut ramp up as recession fears grow']

#### Price Information

In [129]:
hs_yest.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,335.000000,335.000000,335.000000,335.000000,335.0,335.0,335.0
mean,16711.705819,16716.493622,16706.127192,16711.308390,0.0,0.0,0.0
std,58.687718,59.628674,56.715498,58.163696,0.0,0.0,0.0
min,16614.060547,16618.890625,16611.140625,16614.960938,0.0,0.0,0.0
25%,16672.105469,16676.205078,16668.330078,16672.729492,0.0,0.0,0.0
50%,16696.550781,16701.509766,16689.939453,16696.169922,0.0,0.0,0.0
75%,16732.525391,16736.464844,16724.569336,16729.300781,0.0,0.0,0.0
max,16903.240234,16908.970703,16877.949219,16906.250000,0.0,0.0,0.0


In [130]:
plot_price(hs_yest, "Hang Sang Index", "$HKD")